In [1]:
import numpy as np
import time
import scipy

In [23]:
def maxvol1(A_func, I, J, U = None, V = None, maxit=7):
    row = I[np.random.randint(0, len(I))]
    if U is None:
        for it in range(1, maxit+1, 2):
            col = J[np.argmax(np.abs(A_func(row, I)))]
            row = I[np.argmax(np.abs(A_func(J, col)))]
    else:
        for it in range(1, maxit+1, 2):
            col = J[np.argmax(np.abs(A_func(row, J) - U[row, :] @ V[:, J]))]
            row = I[np.argmax(np.abs(A_func(I, col) - U[I, :] @ V[:, col]))]
    return row, col

In [24]:
def mCross(A_func, shape, eps = 1e-5):
    n, m = shape
    I = np.arange(n)
    J = np.arange(m)
    i, j = maxvol1(A_func, I, J)
    U = (A_func(np.arange(n), j) / np.abs(A_func(i, j))**0.5).reshape(-1, 1)
    V = (A_func(i, np.arange(m)) * np.abs(A_func(i, j))**0.5 / A_func(i, j)).reshape(1, -1)
    I = np.setdiff1d(I, [i])
    J = np.setdiff1d(J, [j])
    r = 1
    UVnorm2 = np.linalg.norm(U @ V)**2
    while True:
        i, j = maxvol1(A_func, I, J, U, V)
        UV_ij = U[i, :] @ V[:, j]
        if np.abs(A_func(i, j) - UV_ij) * np.sqrt((m-r) * (n-r)) < eps * np.sqrt(UVnorm2):
            break
        u = (A_func(np.arange(n), j) - U @ V[:, j]) / np.sqrt(np.abs(A_func(i, j) - UV_ij))
        v = (A_func(i,  np.arange(m)) - U[i, :] @ V) * np.sqrt(np.abs(A_func(i, j) - UV_ij)) / (A_func(i, j) - UV_ij)
        UVnorm2 = UVnorm2 + np.linalg.norm(u)**2 * np.linalg.norm(v)**2 + np.dot(U.T @ u, V @ v.T) + np.dot(V @ v.T, U.T @ u)
        U = np.concatenate((U, u.reshape(-1, 1)), axis=1)
        V = np.concatenate((V, v.reshape(1, -1)), axis=0)
        I = np.setdiff1d(I, [i])
        J = np.setdiff1d(J, [j])
        r += 1
    return (U, V, r)

In [18]:
def Hilbert(i, j):
    return 1 / (i + j + 1)

M = 4096
N = 4096

A = np.fromfunction(Hilbert, [M, N])

In [11]:
start = time.time()
u, s, v = np.linalg.svd(A, full_matrices=False)
end = time.time()
print("Classical time: ", end - start, " sec")

Classical time:  15.15151071548462  sec


In [22]:
eps = 1e-6

norm = np.sum(s)
cur_norm = norm.copy()
r = 0
while cur_norm >= eps:
    cur_norm -= s[r]
    r += 1
r, np.sum(s) - np.sum(s[: r])

(16, 4.829631361502607e-07)

In [25]:
start = time.time()
U, V, r = mCross(Hilbert,(N, M), eps)
np.linalg.norm(A - U @ V), r
end = time.time()
print("Cross time: ", end - start, " sec")
print("Rank = {}, Relative approximation error = {}".format(r, np.linalg.norm(A - U @ V) / np.linalg.norm(A)))

Cross time:  0.10158896446228027  sec
Rank = 17, Relative approximation error = 1.6080917904539608e-07
